In [1]:
import pandas as pd
import datetime
import numpy as np
from dateutil.relativedelta import relativedelta

## CDI

In [2]:
df = pd.read_excel("datalake_files/DCF - RP'22 8&04 - Valores.xlsx",sheet_name = 'Budget',header = 5)
df = df.loc[~df['Unnamed: 1'].isnull()].drop('Unnamed: 0',axis = 1).set_index('Unnamed: 1').T
df.columns.name = None
df = df[list(map(lambda x: type(x) == datetime.datetime,df.index.values))]
serie = df['Cash Accumulated Available'] * 100

In [2]:
def composed_interest(days,values):
    results = [values[0] + 1]
    for day,valor in zip(days[1:],values[1:]):
        if day == 0:
            results.append(1)
        else:
            results.append(results[-1] * valor + 1)
    return np.array(results) - 1

In [8]:
arquivo = 'datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx'
dfs = []
messages = []
for i in range(1,11):
    try:
        df = pd.read_excel(arquivo,sheet_name = f'{i}ª Emissão CDI',header = 4)
        df = df[df['Filtro'] == 'X'][['Data','Last  CDI','Days','Interest (Month)','Principal Balance']].dropna()
        df['Data'] = pd.to_datetime(df['Data'],format = '%d%m%Y')
        if df['Data'].iloc[0] < datetime.datetime.now():
            df['Last  CDI'] = df['Last  CDI'] / 100
            spread = pd.read_excel(arquivo,sheet_name = '1ª Emissão CDI').iloc[2,9] / 100
            df['Spread'] = spread
            df['Taxa Diária Estimada'] = ((df['Last  CDI'] + spread + 1) ** (1/252)) - 1
            df['Days Dif'] = df['Days'].diff().fillna(df['Days'].iloc[0]).apply(lambda x: 0 if x < 2 else x)
            df['Taxa Estimada Período'] = ((1 + df['Taxa Diária Estimada']) ** df['Days Dif']) - 1
            df['Taxa Acumulada Estimada'] = composed_interest(df['Days Dif'].values,df['Taxa Estimada Período'].values)
            df['Juros Estimados'] = df['Principal Balance'] * df['Taxa Acumulada Estimada']
            df['Despesa Estimada'] = df['Juros Estimados'].cumsum()
            df['Despesa Real'] = (df['Interest (Month)'] * df['Days Dif'].apply(lambda x: 0 if x == 0 else 1)).cumsum()
            df['Erro'] = 100 * (df['Despesa Real'] - df['Despesa Estimada']) / df['Despesa Real']
            df['Período'] = df['Data'].apply(lambda x: str(x.year) + '-' + str(x.month))
            dfs.append(df)
            messages.append(f'{i}ª Emissão CDI: Sucesso')
        else:
            messages.append(f'{i}ª Emissão CDI: Não Realizado')
    except ValueError as e:
        print(e)
        messages.append(f'{i}ª Emissão CDI: ' + str(e))

Worksheet named '9ª Emissão CDI' not found
Worksheet named '10ª Emissão CDI' not found


## IPCA

In [22]:
arquivo = 'datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx'
i = 1
positions = (15,14)
emissao = i + 1
df = pd.read_excel(arquivo,sheet_name = f'{emissao}ª Emissão IPCA',header = 6)
df = df[df['Filtro'] == 'X'][['Date','IPCA','Interest','Monetary Variation',' Principal Balance']]
df['Date'] = pd.to_datetime(df['Date'],format = '%d%m%Y')
raw = pd.read_excel(arquivo,sheet_name = f'{emissao}ª Emissão IPCA')
total = raw.iloc[0,positions[i]]
juros = raw.iloc[4,positions[i]]
del(raw)
df['Taxa'] = juros
df['Capital'] = total
juros_semestral = ((1 + (juros / 100)) ** (1/2)) - 1
df['Days Dif'] = df['Date'].diff().apply(lambda x: x.days).fillna(30)
df = df[df['Days Dif'] <= 1]
df['Monetary Variation'] = df['Monetary Variation'].fillna(0)
ipca_base = (df['Capital'].iloc[0] / df[' Principal Balance'].iloc[0]) * df['IPCA'].iloc[0]
subtract = 0
for i in df.index:
    if df.loc[i,'Days Dif'] == 1:
        subtract += df.loc[i,'Interest']
    df.loc[i,'Capital'] -= subtract
df = df[df['Days Dif'] != 1]
df['Despesa Acumulada'] = (df['Interest'] + df['Monetary Variation']).cumsum()
df['Capital Dif'] = df['Capital'].diff().fillna(0).cumsum()
df['Principal Balance Calculado'] = (total * df['IPCA'] / ipca_base) + (df['Capital Dif'] * df['IPCA'] / ipca_base)
df['Interest Calculado'] = df['Principal Balance Calculado'] * juros_semestral
faltante = (total + sum(df['Capital'].diff().dropna())) * (-1)
df['Monetary Variation Calculado'] = df['Capital'].diff().apply(lambda x: None if x == 0 else x).fillna(method = 'bfill').fillna(faltante) * (-1) * ((df['IPCA'] / ipca_base) - 1) * df['Monetary Variation'].apply(lambda x: 1 if x > 0 else 0)
df['Despesa Acumulada Calculado'] = (df['Interest Calculado'] + df['Monetary Variation Calculado']).cumsum()
df['Erro'] = 100 * (df['Despesa Acumulada'] - df['Despesa Acumulada Calculado']) / df['Despesa Acumulada']
df

,Date,IPCA,Interest,Monetary Variation,Principal Balance,Taxa,Capital,Days Dif,Despesa Acumulada,Capital Dif,Principal Balance Calculado,Interest Calculado,Monetary Variation Calculado,Despesa Acumulada Calculado,Erro
194,2021-12-15,6075.690000,1.521793e+07,0.000000e+00,6.838447e+08,4.6279,650000000,0.0,1.521793e+07,0.0,6.838447e+08,1.564486e+07,0.000000e+00,1.564486e+07,-2.805450
377,2022-06-15,6412.880000,1.659511e+07,0.000000e+00,7.236822e+08,4.6279,650000000,0.0,3.181305e+07,0.0,7.217969e+08,1.651313e+07,0.000000e+00,3.215799e+07,-1.084286
561,2022-12-15,6425.219033,1.662705e+07,0.000000e+00,7.249103e+08,4.6279,650000000,0.0,4.844010e+07,0.0,7.231857e+08,1.654490e+07,0.000000e+00,4.870289e+07,-0.542515
744,2023-06-15,6651.976786,1.693756e+07,0.000000e+00,7.509255e+08,4.6279,650000000,0.0,6.537766e+07,0.0,7.487082e+08,1.712880e+07,0.000000e+00,6.583169e+07,-0.694480
928,2023-12-15,6796.731464,1.772963e+07,0.000000e+00,7.674342e+08,4.6279,650000000,0.0,8.310728e+07,0.0,7.650010e+08,1.750154e+07,0.000000e+00,8.333323e+07,-0.271876
1114,2024-06-17,6935.142549,1.765857e+07,0.000000e+00,7.832906e+08,4.6279,650000000,0.0,1.007659e+08,0.0,7.805798e+08,1.785795e+07,0.000000e+00,1.011912e+08,-0.422100
1297,2024-12-16,7059.911520,1.870953e+07,0.000000e+00,7.973826e+08,4.6279,650000000,0.0,1.194754e+08,0.0,7.946230e+08,1.817923e+07,0.000000e+00,1.193704e+08,0.087860
1480,2025-06-16,7195.573136,1.817229e+07,0.000000e+00,8.127258e+08,4.6279,650000000,0.0,1.376477e+08,0.0,8.098923e+08,1.852856e+07,0.000000e+00,1.378990e+08,-0.182564
1663,2025-12-15,7335.605691,1.944015e+07,0.000000e+00,8.284212e+08,4.6279,650000000,0.0,1.570878e+08,0.0,8.256536e+08,1.888914e+07,0.000000e+00,1.567881e+08,0.190795
1846,2026-06-15,7470.803381,1.871230e+07,0.000000e+00,8.439015e+08,4.6279,650000000,0.0,1.758001e+08,0.0,8.408707e+08,1.923727e+07,0.000000e+00,1.760254e+08,-0.128134


In [33]:
arquivo = 'datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx'
dfs = []
for i in range(2):
        df = pd.read_excel(arquivo,sheet_name = f'{i}ª Emissão IPCA',header = 6)
        

KeyError: 'Filtro'

## Câmbio

In [13]:
df = pd.read_excel('datalake_files/09 Debt vs Setembro 2022 - RP - Budget 2023.xlsx',sheet_name = 'Daily Calculation prop2019',header = 2)
df = df[['Date','USD','Repayment']].dropna()
df = df.set_index('Date')
df = df[datetime.datetime.today():]
df['Period'] = df.index.to_period('m')
df['Reais_Acumulado'] = (df['USD'] * df['Repayment']).cumsum()